In [1]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from sympy import *
from sympy.abc import *

init_printing()

In [2]:
# 単位ベクトルとか角度の定義
angles = symbols("phi_0 phi_1 phi_2")# 原点から肩部への方向を表す角度
thetas = symbols("theta_0 theta_1 theta_2")# モーターの角度(±90deg)
unit_vectors = [Matrix([cos(angles[i]),sin(angles[i]),0]) for i in range(3)]
ez = Matrix([0,0,1])

In [3]:
# パラメータ(実数)
params = [(A,130.0),(B,200.0),(C,400.0),(D,130.0)]
params.append((angles[0],2.0*np.pi/3.0*0))
params.append((angles[1],2.0*np.pi/3.0*1))
params.append((angles[2],2.0*np.pi/3.0*2))
params.append((x,0))
params.append((y,0))
params.append((z,400))

In [4]:
# 各座標の計算
A_vectors = map(lambda x: A*x,unit_vectors)
B_vectors = [A_vectors[i] + B*(unit_vectors[i] * cos(thetas[i])-ez*sin(thetas[i]) ) for i in range(3)]
D_vector = Matrix([x,y,z])
C_vectors = [D_vector + D * unit_vectors[i] for i in range(3)]

In [5]:
# モーターのある位置の座標．肩っぽい部分の座標．
A_vectors

⎡⎡A⋅cos(φ₀)⎤, ⎡A⋅cos(φ₁)⎤, ⎡A⋅cos(φ₂)⎤⎤
⎢⎢         ⎥  ⎢         ⎥  ⎢         ⎥⎥
⎢⎢A⋅sin(φ₀)⎥  ⎢A⋅sin(φ₁)⎥  ⎢A⋅sin(φ₂)⎥⎥
⎢⎢         ⎥  ⎢         ⎥  ⎢         ⎥⎥
⎣⎣    0    ⎦  ⎣    0    ⎦  ⎣    0    ⎦⎦

In [6]:
# 腕の中央の関節．肘っぽい部分の座標
B_vectors

⎡⎡A⋅cos(φ₀) + B⋅cos(φ₀)⋅cos(θ₀)⎤, ⎡A⋅cos(φ₁) + B⋅cos(φ₁)⋅cos(θ₁)⎤, ⎡A⋅cos(φ₂) 
⎢⎢                             ⎥  ⎢                             ⎥  ⎢          
⎢⎢A⋅sin(φ₀) + B⋅sin(φ₀)⋅cos(θ₀)⎥  ⎢A⋅sin(φ₁) + B⋅sin(φ₁)⋅cos(θ₁)⎥  ⎢A⋅sin(φ₂) 
⎢⎢                             ⎥  ⎢                             ⎥  ⎢          
⎣⎣         -B⋅sin(θ₀)          ⎦  ⎣         -B⋅sin(θ₁)          ⎦  ⎣         -

+ B⋅cos(φ₂)⋅cos(θ₂)⎤⎤
                   ⎥⎥
+ B⋅sin(φ₂)⋅cos(θ₂)⎥⎥
                   ⎥⎥
B⋅sin(θ₂)          ⎦⎦

In [7]:
# 下部の接続部分の座標．手首みたいな場所，
C_vectors

⎡⎡D⋅cos(φ₀) + x⎤, ⎡D⋅cos(φ₁) + x⎤, ⎡D⋅cos(φ₂) + x⎤⎤
⎢⎢             ⎥  ⎢             ⎥  ⎢             ⎥⎥
⎢⎢D⋅sin(φ₀) + y⎥  ⎢D⋅sin(φ₁) + y⎥  ⎢D⋅sin(φ₂) + y⎥⎥
⎢⎢             ⎥  ⎢             ⎥  ⎢             ⎥⎥
⎣⎣      z      ⎦  ⎣      z      ⎦  ⎣      z      ⎦⎦

In [8]:
# 下部の中央部分の座標．
D_vector

⎡x⎤
⎢ ⎥
⎢y⎥
⎢ ⎥
⎣z⎦

In [9]:
def norm2(x):
    """ 3次元ベクトルxのノルムの2乗を計算する関数 """
    return x[0]*x[0]+x[1]*x[1]+x[2]*x[2]

In [10]:
# 肘部分と手首の距離はCとして定義していることから
# (C_vectors[0] - Bvectors[0]).norm() - C == 0 となります
# ただ標準のnormではうまく計算ができなかったので自作した関数を用いてノルム(の2乗)を計算します
eq0 = C**2-norm2((C_vectors[0] - B_vectors[0]))
eq0 = expand(eq0)
eq0 = simplify(eq0)
eq0

In [11]:
# 先に P + Q * sin(theta_0) + R * cos(theta_0) = 0 の方程式を考える
formula = Eq(P + Q * sin(thetas[0]) + R * cos(thetas[0]),0)
formula

In [12]:
# thetaについての解はこうなる
theta_ans = solve(formula,thetas[0])

In [13]:
# P + Q*sin(theta_0) + R*cos(theta_0) = 0 の形を取り出す
Q_dash = eq0.coeff(sin(thetas[0]),1)
R_dash = eq0.coeff(cos(thetas[0]),1)
P_dash = eq0.coeff(cos(thetas[0]),0) - Q_dash * sin(thetas[0])

# P + Q*sin(thetas[0])+R*cos(thetas[0])が上のeq0と同じものであるか確認
simplify(expand(P_dash + Q_dash*sin(thetas[0])+R_dash*cos(thetas[0]))) 

In [14]:
P_dash

In [15]:
Q_dash

In [16]:
R_dash

In [17]:
# Pythonのコードを生成
from sympy.printing.lambdarepr import lambdarepr
print("P = " + lambdarepr(P_dash))
print("Q = " + lambdarepr(Q_dash))
print("R = " + lambdarepr(R_dash))
print("theta_0 = " + lambdarepr(theta_ans[0]))
print("theta_1 = " + lambdarepr(theta_ans[1]))

P = -A**2 + 2*A*D + 2*A*x*cos(phi_0) + 2*A*y*sin(phi_0) - B**2 + C**2 - D**2 - 2*D*x*cos(phi_0) - 2*D*y*sin(phi_0) - x**2 - y**2 - z**2
Q = -2*B*z
R = -2*A*B + 2*B*D + 2*B*x*cos(phi_0) + 2*B*y*sin(phi_0)
theta_0 = -2*atan((Q - sqrt(-P**2 + Q**2 + R**2))/(P - R))
theta_1 = -2*atan((Q + sqrt(-P**2 + Q**2 + R**2))/(P - R))


In [18]:
# Cのコードを生成
float_type = "double"
print(float_type + " P = " + ccode(P_dash) + ";")
print(float_type + " Q = " + ccode(Q_dash) + ";")
print(float_type + " R = " + ccode(R_dash) + ";")
print(float_type + " theta_0 = " + ccode(theta_ans[0]) + ";")
print(float_type + " theta_1 = " + ccode(theta_ans[1]) + ";")

double P = -pow(A, 2) + 2*A*D + 2*A*x*cos(phi_0) + 2*A*y*sin(phi_0) - pow(B, 2) + pow(C, 2) - pow(D, 2) - 2*D*x*cos(phi_0) - 2*D*y*sin(phi_0) - pow(x, 2) - pow(y, 2) - pow(z, 2);
double Q = -2*B*z;
double R = -2*A*B + 2*B*D + 2*B*x*cos(phi_0) + 2*B*y*sin(phi_0);
double theta_0 = -2*atan((Q - sqrt(-pow(P, 2) + pow(Q, 2) + pow(R, 2)))/(P - R));
double theta_1 = -2*atan((Q + sqrt(-pow(P, 2) + pow(Q, 2) + pow(R, 2)))/(P - R));
